# Aggregating and combining metrics

The plot metrics we previously created can be 0D (single value), 1D (vertical profile), 2D (single values across an xy grid) or 3D (vertical profiles across an xy grid - ie. voxels). 

At the moment, to compare plots we just want 0D metrics.

For the non gridded metrics, we've already calculated some summary stats over the vertical profiles, so we can just drop those metrics / data_vars. Similarly, we can drop the vertical profiles in the gridded metrics to just have 2D metrics. Now we can summarise across the x and y dimensions using something like sd or cv to capture variability. 

Not using open_mfdataset this time because it seems to be very slow.

In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
data_dir = Path("../data")
plots_dir = data_dir / "outputs" / "plots"
metrics_dir = plots_dir / "metrics"
lidar_dir = plots_dir / "lidar"

In [3]:
def drop_z_data_vars(ds):
    vars_without_z = [v for v in ds.data_vars if "z" not in ds[v].dims]
    ds = ds[vars_without_z]
    return ds


def read_metrics(plot: str, subdir="no_grid_z_1m"):
    path = metrics_dir / subdir / f"{plot}.nc"
    with xr.open_dataset(path) as ds:
        return ds.load()


def ds_to_df(ds, grid_size=0):
    ds = drop_z_data_vars(ds)

    if grid_size > 0:
        ds_mean = ds.mean(dim=["x", "y"])
        ds_std = ds.std(dim=["x", "y"])
        ds_cv = ds_std / ds_mean

        df_mean = ds_mean.stack(plot_variant=("plot", "variant")).to_dataframe()
        df_std = ds_std.stack(plot_variant=("plot", "variant")).to_dataframe()
        df_cv = ds_cv.stack(plot_variant=("plot", "variant")).to_dataframe()

        df_mean["grid_agg"] = "mean"
        df_std["grid_agg"] = "std"
        df_cv["grid_agg"] = "cv"
        df = pd.concat([df_mean, df_std, df_cv])
    else:
        df = ds.stack(plot_variant=("plot", "variant")).to_dataframe()
        df["grid_agg"] = 'none'

    df["grid_size"] = grid_size

    # Drop the plot_variant multi-index
    df = df.reset_index(drop=True)
    df = df.set_index("plot")

    return df


def load_df_for_grid_size(plot: str, grid_size=0):
    if grid_size == 0:
        ds = read_metrics(plot, "no_grid_z_1m")
    elif grid_size == 10:
        ds = read_metrics(plot, "grid_10m_z_1m")
    elif grid_size == 5:
        ds = read_metrics(plot, "grid_5m_z_1m")
    else:
        raise ValueError(f"Grid size {grid_size} not supported")

    return ds_to_df(ds, grid_size)


def load_plot_metrics(plot: str):
    df_no_grid = load_df_for_grid_size(plot, 0)
    df_grid_10m = load_df_for_grid_size(plot, 10)
    df_grid_5m = load_df_for_grid_size(plot, 5)
    # TODO - 1m

    return pd.concat([df_no_grid, df_grid_10m, df_grid_5m])

In [4]:
plots = gpd.read_file(plots_dir / "plots.geojson")
plot_ids = plots['id'].to_list()

In [5]:
from dask.distributed import Client

client = Client()  # Start a Dask client
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59447,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:59459,Total threads: 2
Dashboard: http://127.0.0.1:59464/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:59450,


In [6]:
futures = client.map(load_plot_metrics, plot_ids, key=plot_ids)

In [7]:
results = client.gather(futures, errors='skip')

In [8]:
df = pd.concat(results)
df

,max,min,range,mean,median,sd,var,cv,crr,skew,...,norm_fhd,cv_inside,cv_inside_p,cv_ppi,cv_vad,site,site_type,variant,grid_agg,grid_size
plot,,,,,,,,,,,,,,,,,,,,,
AGG_O_01_P1,16.275999,0.000000,16.275999,4.428016,4.555000,4.093416,16.756056,0.924436,0.272058,0.245465,...,0.824705,1.051162,1.051162,0.111321,1.511392,AGG_O_01,AGG,default,none,0
AGG_O_01_P1,16.275999,0.000000,16.275999,6.716327,7.568000,3.647536,13.304515,0.543085,0.412652,-0.601755,...,0.860111,0.848656,0.848656,0.129149,0.997475,AGG_O_01,AGG,rnw,none,0
AGG_O_01_P1,16.275999,0.000000,16.275999,4.428016,4.555000,4.093416,16.756056,0.924436,0.272058,0.245465,...,0.811571,1.125259,1.125259,0.109713,1.573446,AGG_O_01,AGG,fr,none,0
AGG_O_01_P1,16.275999,0.001000,16.275000,5.646877,6.400000,3.805994,14.485587,0.674000,0.346905,-0.170667,...,0.838153,0.972110,0.972110,0.070207,0.761257,AGG_O_01,AGG,v0,none,0
AGG_O_01_P1,13.216293,0.000000,13.216293,4.163065,4.232939,3.782385,14.912291,0.941273,0.314709,0.265488,...,0.836686,0.998739,0.998739,0.135513,1.434333,AGG_O_01,AGG,default,mean,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ULY_Y_96_P5,6.922571,0.040224,6.929721,2.038048,1.393568,2.467174,26.531130,0.432079,0.110644,1.652228,...,0.188681,0.619318,0.619318,0.043333,0.506355,ULY_Y_96,ULY,v0,std,5
ULY_Y_96_P5,0.615217,5.962366,0.616002,0.926359,1.611275,0.879022,1.828073,0.336213,0.627924,0.709468,...,0.206563,0.332488,0.332488,0.481339,0.366383,ULY_Y_96,ULY,default,cv,5
ULY_Y_96_P5,0.615217,7.371727,0.617074,0.978764,1.852906,0.857200,1.739255,0.377649,0.699661,0.791740,...,0.216635,0.355344,0.355344,0.479070,0.380383,ULY_Y_96,ULY,rnw,cv,5


In [10]:
df.to_csv(plots_dir /  "plot_metrics.csv")
df.to_parquet(plots_dir / "plot_metrics.parquet")
df.to_json(plots_dir / "plot_metrics.json", orient='records')

In [14]:
client.close()